In [1]:
import pandas as pd

# Read in dataset
To my knowledge, I only need pandas, since this is only cleaning and not analyzing/plotting

In [2]:
data = pd.read_csv("C:\\Users\\bkb3\\Documents\\US_Accidents_March23.csv")


In [3]:
# Create one hot encoding function for categorical columns
def onehotinator(df, string):
    one_hot = pd.get_dummies(df[string], prefix=string)
    one_hot = one_hot.astype(int)
    df = df.join(one_hot)
    df = df.drop(columns=string)
    return df

# Step 1: Check Missing Data
This dataset is huge, so I am not yet too worried about losing data. I mostly want to see which specific columns will need to be dropped/filled in as 0 right off the bat

In [4]:
data.isnull().sum()

ID                             0
Source                         0
Severity                       0
Start_Time                     0
End_Time                       0
Start_Lat                      0
Start_Lng                      0
End_Lat                  3402762
End_Lng                  3402762
Distance(mi)                   0
Description                    5
Street                     10869
City                         253
County                         0
State                          0
Zipcode                     1915
Country                        0
Timezone                    7808
Airport_Code               22635
Weather_Timestamp         120228
Temperature(F)            163853
Wind_Chill(F)            1999019
Humidity(%)               174144
Pressure(in)              140679
Visibility(mi)            177098
Wind_Direction            175206
Wind_Speed(mph)           571233
Precipitation(in)        2203586
Weather_Condition         173459
Amenity                        0
Bump      

# Step 2: Subset Maryland
No sense trying to clean yet; the vast majority is not needed. I first needed to see how "Maryland" was spelled as an entry in the state column: MD. After finding this, I subset out only all rows from the state of Maryland. The rest of the bits of code here is me just checking I didn't accidentally do something incorrect in execution, even if the code ran without error

In [5]:
# Find MD spelling
state_list = data['State'].unique()
state_list

# Subset only md data
df = data[data["State"]=='MD']
df

# Confirm correct execution
df['State'].unique()

# Remove state col cause its only md
df = df.drop(columns='State')
df

# Confirm correct execution
print(df.shape)

(140417, 45)


In [6]:
# Making sure to do the best practice of dropping duplicate rows
df = df.drop_duplicates()

# Step 3: Deal with MD Null Values
Now that we only have the data we will actually work with, I can begin to figure out how to fully clean the data. My first instinct is to look at which columns still have missing values and prioritize dealing with them before anything else: data type conversion comes to mind as well. Looking at this dataset, quite a few columns are still missing values. However, I noticed that they tended to fall into 2 camps: missing a few hundred entries, and missing over 10000 entries. I decided to start by removing all rows where every value is missing and see what happened.

In [7]:
# See if any rows are fully null values
df = df.dropna(how='all')
df.isnull().sum()
print(df.shape)

(140417, 45)


# Step 4: Actually deal with MD Null Values
Since it seems that none of the rows of data are fully empty, I wanted to compare to a nuclear option: delete all rows with at least one missing value, and see what happened.

In [8]:
# Drop any rows with any null values
df_drop_any = df.dropna(how='any')
print(df.shape, df_drop_any.shape)

(140417, 45) (66544, 45)


# Step 5: Find a less Destructive way to deal with missing values
Given that how=any knocked out half the data, I wanted to instead backtrack, and see if I could eliminate missing values by simply deleting columns half empty, assuming I could justify not needing them. The question then became: What exactly counts as "necessary?" I started with a safe bet of removing the end lat/lng columns; the start lat/lng columns are fully filled in, and frankly I am not sure what use a half empty "end crash location" set of columns is off hand.

In [9]:
# Check MD Null Values
df.isnull().sum()
# Remove end lat/lon, not really sure how to reconcile those outside of removal
df = df.drop(columns=["End_Lat","End_Lng"])
# Check MD Null Values
df.isnull().sum()


ID                           0
Source                       0
Severity                     0
Start_Time                   0
End_Time                     0
Start_Lat                    0
Start_Lng                    0
Distance(mi)                 0
Description                  0
Street                     363
City                         9
County                       0
Zipcode                    260
Country                      0
Timezone                   260
Airport_Code               327
Weather_Timestamp        13519
Temperature(F)           15713
Wind_Chill(F)            47596
Humidity(%)              15827
Pressure(in)             14528
Visibility(mi)           27461
Wind_Direction           20204
Wind_Speed(mph)          34300
Precipitation(in)        49483
Weather_Condition        26888
Amenity                      0
Bump                         0
Crossing                     0
Give_Way                     0
Junction                     0
No_Exit                      0
Railway 

# Well....That's Just MD
After subsetting to only MD, I found that several columns now only had 1 entry, all of which were obvious. So, I deleted those out of hand.

In [10]:
# Remove columns with only one type of entry    
df = df.drop(columns=["Country", "Timezone", "Turning_Loop"])
for x in df.columns:
    print(df[x].nunique())


140417
3
4
117296
126300
50355
50643
6274
69032
7621
381
31
10859
34
70648
523
517
95
279
40
24
67
88
71
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


To be completely honest, in the cases of columns with tens of thousands of missing values, I am not fully certain what the best practice would be. However, I wanted to see if I could try to fix the columns with only a few hundred missing values.

In [11]:
df.columns
# Check MD Null Values
df.isnull().sum()
# Drop columns with 10000+ null values
df = df.drop(columns=['Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 
                        'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition'])

df.isnull().sum()

ID                         0
Source                     0
Severity                   0
Start_Time                 0
End_Time                   0
Start_Lat                  0
Start_Lng                  0
Distance(mi)               0
Description                0
Street                   363
City                       9
County                     0
Zipcode                  260
Airport_Code             327
Amenity                    0
Bump                       0
Crossing                   0
Give_Way                   0
Junction                   0
No_Exit                    0
Railway                    0
Roundabout                 0
Station                    0
Stop                       0
Traffic_Calming            0
Traffic_Signal             0
Sunrise_Sunset           286
Civil_Twilight           286
Nautical_Twilight        286
Astronomical_Twilight    286
dtype: int64

In [12]:
# Drop rows with any null values present
df = df.dropna(how='any')
df.isnull().sum()

# Pivot to changing datatypes from cat/bool to int - figuring out which columns are bool first
df.dtypes


ID                        object
Source                    object
Severity                   int64
Start_Time                object
End_Time                  object
Start_Lat                float64
Start_Lng                float64
Distance(mi)             float64
Description               object
Street                    object
City                      object
County                    object
Zipcode                   object
Airport_Code              object
Amenity                     bool
Bump                        bool
Crossing                    bool
Give_Way                    bool
Junction                    bool
No_Exit                     bool
Railway                     bool
Roundabout                  bool
Station                     bool
Stop                        bool
Traffic_Calming             bool
Traffic_Signal              bool
Sunrise_Sunset            object
Civil_Twilight            object
Nautical_Twilight         object
Astronomical_Twilight     object
dtype: obj

Need to do the following:
Plug in each bool column from dataframe
Change bool format to int format without making new column

In [13]:

# Blueprint to change True/False columns to 1/0

to_change = ['Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 
             'Traffic_Signal']

df = df.astype({'Amenity': int, 'Bump': int, 'Crossing': int, 'Give_Way': int, 'Junction': int, 'No_Exit': int, 
                              'Railway': int, 'Roundabout': int, 'Station': int, 'Stop': int, 'Traffic_Calming': int, 'Traffic_Signal': int}) 

df





,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Street,...,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
228546,A-228553,Source2,2,2016-11-30 11:58:01,2016-11-30 16:57:20,39.152046,-76.645584,0.010,Accident on MD-100 Eastbound at Exits 13A 13A-...,I-97 S,...,0,0,0,0,0,0,Day,Day,Day,Day
228547,A-228554,Source2,3,2016-11-30 12:04:03,2016-11-30 17:02:36,39.152046,-76.645584,0.010,Lane blocked and right hand shoulder blocked d...,I-97 S,...,0,0,0,0,0,0,Day,Day,Day,Day
228560,A-228567,Source2,2,2016-11-30 16:54:53,2016-11-30 17:24:12,39.481083,-76.249626,0.010,Accident on MD-7 Philadelphia Rd at MD-543 Riv...,Philadelphia Rd,...,0,0,0,0,0,1,Night,Day,Day,Day
228567,A-228574,Source2,2,2016-11-30 17:33:47,2016-11-30 18:01:49,39.158756,-76.718475,0.010,Accident on MD-100 near Exits 10 10A 10B MD-71...,MD-100 E,...,0,0,0,0,0,0,Night,Night,Day,Day
228573,A-228580,Source2,2,2016-11-30 17:35:33,2016-11-30 18:05:10,39.636391,-77.710457,0.010,Accident on Cleveland Ave around Antietam St.,E Antietam St,...,0,0,0,0,0,1,Night,Night,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7727511,A-7776879,Source1,2,2019-08-22 19:48:26,2019-08-22 20:17:11,39.494560,-76.892370,0.084,At MD-140/Baltimore Blvd - Accident. Lane bloc...,Gamber Rd,...,0,0,0,0,0,0,Day,Day,Day,Day
7727512,A-7776880,Source1,2,2019-08-22 19:48:26,2019-08-22 20:17:11,39.495610,-76.891580,0.084,At MD-140/Baltimore Blvd - Accident. Lane bloc...,Emory Rd,...,0,0,0,0,0,1,Day,Day,Day,Day
7727513,A-7776881,Source1,4,2019-08-22 19:57:25,2019-08-23 00:56:06,39.600500,-77.677230,3.053,Closed between MD-66/Exit 35 and MD-65/Exit 29...,I-70 W,...,0,0,0,0,0,0,Night,Day,Day,Day
7727996,A-7777364,Source1,2,2019-08-23 08:03:29,2019-08-23 08:32:31,39.336120,-76.491730,0.273,At MD-7/Exit 34 - Multi-vehicle accident invol...,I-695 N,...,0,0,0,0,0,0,Day,Day,Day,Day


In [14]:
# Referencing columns datatypes above, change day/night columns to day=1, night=0

to_change2 = ['Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']
df['Sunrise_Sunset'] = df.Sunrise_Sunset.map({'Day':1,'Night':0}) 
df['Civil_Twilight'] = df.Civil_Twilight.map({'Day':1,'Night':0}) 
df['Nautical_Twilight'] = df.Nautical_Twilight.map({'Day':1,'Night':0}) 
df['Astronomical_Twilight'] = df.Astronomical_Twilight.map({'Day':1,'Night':0}) 
df

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Street,...,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
228546,A-228553,Source2,2,2016-11-30 11:58:01,2016-11-30 16:57:20,39.152046,-76.645584,0.010,Accident on MD-100 Eastbound at Exits 13A 13A-...,I-97 S,...,0,0,0,0,0,0,1,1,1,1
228547,A-228554,Source2,3,2016-11-30 12:04:03,2016-11-30 17:02:36,39.152046,-76.645584,0.010,Lane blocked and right hand shoulder blocked d...,I-97 S,...,0,0,0,0,0,0,1,1,1,1
228560,A-228567,Source2,2,2016-11-30 16:54:53,2016-11-30 17:24:12,39.481083,-76.249626,0.010,Accident on MD-7 Philadelphia Rd at MD-543 Riv...,Philadelphia Rd,...,0,0,0,0,0,1,0,1,1,1
228567,A-228574,Source2,2,2016-11-30 17:33:47,2016-11-30 18:01:49,39.158756,-76.718475,0.010,Accident on MD-100 near Exits 10 10A 10B MD-71...,MD-100 E,...,0,0,0,0,0,0,0,0,1,1
228573,A-228580,Source2,2,2016-11-30 17:35:33,2016-11-30 18:05:10,39.636391,-77.710457,0.010,Accident on Cleveland Ave around Antietam St.,E Antietam St,...,0,0,0,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7727511,A-7776879,Source1,2,2019-08-22 19:48:26,2019-08-22 20:17:11,39.494560,-76.892370,0.084,At MD-140/Baltimore Blvd - Accident. Lane bloc...,Gamber Rd,...,0,0,0,0,0,0,1,1,1,1
7727512,A-7776880,Source1,2,2019-08-22 19:48:26,2019-08-22 20:17:11,39.495610,-76.891580,0.084,At MD-140/Baltimore Blvd - Accident. Lane bloc...,Emory Rd,...,0,0,0,0,0,1,1,1,1,1
7727513,A-7776881,Source1,4,2019-08-22 19:57:25,2019-08-23 00:56:06,39.600500,-77.677230,3.053,Closed between MD-66/Exit 35 and MD-65/Exit 29...,I-70 W,...,0,0,0,0,0,0,0,1,1,1
7727996,A-7777364,Source1,2,2019-08-23 08:03:29,2019-08-23 08:32:31,39.336120,-76.491730,0.273,At MD-7/Exit 34 - Multi-vehicle accident invol...,I-695 N,...,0,0,0,0,0,0,1,1,1,1


In [ ]:
# One hot encode source column, merge to end of dataset, drop original
df = onehotinator(df, 'Source')

# Check columns left over to decide next step
df.columns


Index(['ID', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
       'Distance(mi)', 'Description', 'Street', 'City', 'County', 'Zipcode',
       'Airport_Code', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction',
       'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop',
       'Traffic_Calming', 'Traffic_Signal', 'Sunrise_Sunset', 'Civil_Twilight',
       'Nautical_Twilight', 'Astronomical_Twilight', 'Source_Source1',
       'Source_Source2', 'Source_Source3'],
      dtype='object')

In [16]:
# One hot encode severity column, merge to end of dataset, drop original
df = onehotinator(df, 'Severity')


In [17]:
# Change start time column from string to date time data type
df['Date'] = pd.to_datetime(df['Start_Time'], errors='coerce')
df

,ID,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Street,City,County,...,Nautical_Twilight,Astronomical_Twilight,Source_Source1,Source_Source2,Source_Source3,Severity_1,Severity_2,Severity_3,Severity_4,Date
228546,A-228553,2016-11-30 11:58:01,2016-11-30 16:57:20,39.152046,-76.645584,0.010,Accident on MD-100 Eastbound at Exits 13A 13A-...,I-97 S,Severn,Anne Arundel,...,1,1,0,1,0,0,1,0,0,2016-11-30 11:58:01
228547,A-228554,2016-11-30 12:04:03,2016-11-30 17:02:36,39.152046,-76.645584,0.010,Lane blocked and right hand shoulder blocked d...,I-97 S,Severn,Anne Arundel,...,1,1,0,1,0,0,0,1,0,2016-11-30 12:04:03
228560,A-228567,2016-11-30 16:54:53,2016-11-30 17:24:12,39.481083,-76.249626,0.010,Accident on MD-7 Philadelphia Rd at MD-543 Riv...,Philadelphia Rd,Belcamp,Harford,...,1,1,0,1,0,0,1,0,0,2016-11-30 16:54:53
228567,A-228574,2016-11-30 17:33:47,2016-11-30 18:01:49,39.158756,-76.718475,0.010,Accident on MD-100 near Exits 10 10A 10B MD-71...,MD-100 E,Hanover,Anne Arundel,...,1,1,0,1,0,0,1,0,0,2016-11-30 17:33:47
228573,A-228580,2016-11-30 17:35:33,2016-11-30 18:05:10,39.636391,-77.710457,0.010,Accident on Cleveland Ave around Antietam St.,E Antietam St,Hagerstown,Washington,...,1,1,0,1,0,0,1,0,0,2016-11-30 17:35:33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7727511,A-7776879,2019-08-22 19:48:26,2019-08-22 20:17:11,39.494560,-76.892370,0.084,At MD-140/Baltimore Blvd - Accident. Lane bloc...,Gamber Rd,Finksburg,Carroll,...,1,1,1,0,0,0,1,0,0,2019-08-22 19:48:26
7727512,A-7776880,2019-08-22 19:48:26,2019-08-22 20:17:11,39.495610,-76.891580,0.084,At MD-140/Baltimore Blvd - Accident. Lane bloc...,Emory Rd,Finksburg,Carroll,...,1,1,1,0,0,0,1,0,0,2019-08-22 19:48:26
7727513,A-7776881,2019-08-22 19:57:25,2019-08-23 00:56:06,39.600500,-77.677230,3.053,Closed between MD-66/Exit 35 and MD-65/Exit 29...,I-70 W,Hagerstown,Washington,...,1,1,1,0,0,0,0,0,1,2019-08-22 19:57:25
7727996,A-7777364,2019-08-23 08:03:29,2019-08-23 08:32:31,39.336120,-76.491730,0.273,At MD-7/Exit 34 - Multi-vehicle accident invol...,I-695 N,Rosedale,Baltimore County,...,1,1,1,0,0,0,1,0,0,2019-08-23 08:03:29


In [18]:
# Change end time column from string to date time data type
df['End_Date'] = pd.to_datetime(df['End_Time'], errors='coerce')
df

,ID,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Street,City,County,...,Astronomical_Twilight,Source_Source1,Source_Source2,Source_Source3,Severity_1,Severity_2,Severity_3,Severity_4,Date,End_Date
228546,A-228553,2016-11-30 11:58:01,2016-11-30 16:57:20,39.152046,-76.645584,0.010,Accident on MD-100 Eastbound at Exits 13A 13A-...,I-97 S,Severn,Anne Arundel,...,1,0,1,0,0,1,0,0,2016-11-30 11:58:01,2016-11-30 16:57:20
228547,A-228554,2016-11-30 12:04:03,2016-11-30 17:02:36,39.152046,-76.645584,0.010,Lane blocked and right hand shoulder blocked d...,I-97 S,Severn,Anne Arundel,...,1,0,1,0,0,0,1,0,2016-11-30 12:04:03,2016-11-30 17:02:36
228560,A-228567,2016-11-30 16:54:53,2016-11-30 17:24:12,39.481083,-76.249626,0.010,Accident on MD-7 Philadelphia Rd at MD-543 Riv...,Philadelphia Rd,Belcamp,Harford,...,1,0,1,0,0,1,0,0,2016-11-30 16:54:53,2016-11-30 17:24:12
228567,A-228574,2016-11-30 17:33:47,2016-11-30 18:01:49,39.158756,-76.718475,0.010,Accident on MD-100 near Exits 10 10A 10B MD-71...,MD-100 E,Hanover,Anne Arundel,...,1,0,1,0,0,1,0,0,2016-11-30 17:33:47,2016-11-30 18:01:49
228573,A-228580,2016-11-30 17:35:33,2016-11-30 18:05:10,39.636391,-77.710457,0.010,Accident on Cleveland Ave around Antietam St.,E Antietam St,Hagerstown,Washington,...,1,0,1,0,0,1,0,0,2016-11-30 17:35:33,2016-11-30 18:05:10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7727511,A-7776879,2019-08-22 19:48:26,2019-08-22 20:17:11,39.494560,-76.892370,0.084,At MD-140/Baltimore Blvd - Accident. Lane bloc...,Gamber Rd,Finksburg,Carroll,...,1,1,0,0,0,1,0,0,2019-08-22 19:48:26,2019-08-22 20:17:11
7727512,A-7776880,2019-08-22 19:48:26,2019-08-22 20:17:11,39.495610,-76.891580,0.084,At MD-140/Baltimore Blvd - Accident. Lane bloc...,Emory Rd,Finksburg,Carroll,...,1,1,0,0,0,1,0,0,2019-08-22 19:48:26,2019-08-22 20:17:11
7727513,A-7776881,2019-08-22 19:57:25,2019-08-23 00:56:06,39.600500,-77.677230,3.053,Closed between MD-66/Exit 35 and MD-65/Exit 29...,I-70 W,Hagerstown,Washington,...,1,1,0,0,0,0,0,1,2019-08-22 19:57:25,2019-08-23 00:56:06
7727996,A-7777364,2019-08-23 08:03:29,2019-08-23 08:32:31,39.336120,-76.491730,0.273,At MD-7/Exit 34 - Multi-vehicle accident invol...,I-695 N,Rosedale,Baltimore County,...,1,1,0,0,0,1,0,0,2019-08-23 08:03:29,2019-08-23 08:32:31


In [19]:
# Feature engineer a column of how long the accident occurred in hours
df['Accident_Time_Hr'] = (df.End_Date - df.Date) / pd.Timedelta(hours=1)
df

,ID,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Street,City,County,...,Source_Source1,Source_Source2,Source_Source3,Severity_1,Severity_2,Severity_3,Severity_4,Date,End_Date,Accident_Time_Hr
228546,A-228553,2016-11-30 11:58:01,2016-11-30 16:57:20,39.152046,-76.645584,0.010,Accident on MD-100 Eastbound at Exits 13A 13A-...,I-97 S,Severn,Anne Arundel,...,0,1,0,0,1,0,0,2016-11-30 11:58:01,2016-11-30 16:57:20,4.988611
228547,A-228554,2016-11-30 12:04:03,2016-11-30 17:02:36,39.152046,-76.645584,0.010,Lane blocked and right hand shoulder blocked d...,I-97 S,Severn,Anne Arundel,...,0,1,0,0,0,1,0,2016-11-30 12:04:03,2016-11-30 17:02:36,4.975833
228560,A-228567,2016-11-30 16:54:53,2016-11-30 17:24:12,39.481083,-76.249626,0.010,Accident on MD-7 Philadelphia Rd at MD-543 Riv...,Philadelphia Rd,Belcamp,Harford,...,0,1,0,0,1,0,0,2016-11-30 16:54:53,2016-11-30 17:24:12,0.488611
228567,A-228574,2016-11-30 17:33:47,2016-11-30 18:01:49,39.158756,-76.718475,0.010,Accident on MD-100 near Exits 10 10A 10B MD-71...,MD-100 E,Hanover,Anne Arundel,...,0,1,0,0,1,0,0,2016-11-30 17:33:47,2016-11-30 18:01:49,0.467222
228573,A-228580,2016-11-30 17:35:33,2016-11-30 18:05:10,39.636391,-77.710457,0.010,Accident on Cleveland Ave around Antietam St.,E Antietam St,Hagerstown,Washington,...,0,1,0,0,1,0,0,2016-11-30 17:35:33,2016-11-30 18:05:10,0.493611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7727511,A-7776879,2019-08-22 19:48:26,2019-08-22 20:17:11,39.494560,-76.892370,0.084,At MD-140/Baltimore Blvd - Accident. Lane bloc...,Gamber Rd,Finksburg,Carroll,...,1,0,0,0,1,0,0,2019-08-22 19:48:26,2019-08-22 20:17:11,0.479167
7727512,A-7776880,2019-08-22 19:48:26,2019-08-22 20:17:11,39.495610,-76.891580,0.084,At MD-140/Baltimore Blvd - Accident. Lane bloc...,Emory Rd,Finksburg,Carroll,...,1,0,0,0,1,0,0,2019-08-22 19:48:26,2019-08-22 20:17:11,0.479167
7727513,A-7776881,2019-08-22 19:57:25,2019-08-23 00:56:06,39.600500,-77.677230,3.053,Closed between MD-66/Exit 35 and MD-65/Exit 29...,I-70 W,Hagerstown,Washington,...,1,0,0,0,0,0,1,2019-08-22 19:57:25,2019-08-23 00:56:06,4.978056
7727996,A-7777364,2019-08-23 08:03:29,2019-08-23 08:32:31,39.336120,-76.491730,0.273,At MD-7/Exit 34 - Multi-vehicle accident invol...,I-695 N,Rosedale,Baltimore County,...,1,0,0,0,1,0,0,2019-08-23 08:03:29,2019-08-23 08:32:31,0.483889


In [20]:
# Remove coordinate columns
df = df.drop(columns=['Start_Lat', 'Start_Lng'])
df

,ID,Start_Time,End_Time,Distance(mi),Description,Street,City,County,Zipcode,Airport_Code,...,Source_Source1,Source_Source2,Source_Source3,Severity_1,Severity_2,Severity_3,Severity_4,Date,End_Date,Accident_Time_Hr
228546,A-228553,2016-11-30 11:58:01,2016-11-30 16:57:20,0.010,Accident on MD-100 Eastbound at Exits 13A 13A-...,I-97 S,Severn,Anne Arundel,21144,KBWI,...,0,1,0,0,1,0,0,2016-11-30 11:58:01,2016-11-30 16:57:20,4.988611
228547,A-228554,2016-11-30 12:04:03,2016-11-30 17:02:36,0.010,Lane blocked and right hand shoulder blocked d...,I-97 S,Severn,Anne Arundel,21144,KBWI,...,0,1,0,0,0,1,0,2016-11-30 12:04:03,2016-11-30 17:02:36,4.975833
228560,A-228567,2016-11-30 16:54:53,2016-11-30 17:24:12,0.010,Accident on MD-7 Philadelphia Rd at MD-543 Riv...,Philadelphia Rd,Belcamp,Harford,21017,KAPG,...,0,1,0,0,1,0,0,2016-11-30 16:54:53,2016-11-30 17:24:12,0.488611
228567,A-228574,2016-11-30 17:33:47,2016-11-30 18:01:49,0.010,Accident on MD-100 near Exits 10 10A 10B MD-71...,MD-100 E,Hanover,Anne Arundel,21076,KBWI,...,0,1,0,0,1,0,0,2016-11-30 17:33:47,2016-11-30 18:01:49,0.467222
228573,A-228580,2016-11-30 17:35:33,2016-11-30 18:05:10,0.010,Accident on Cleveland Ave around Antietam St.,E Antietam St,Hagerstown,Washington,21740-5701,KHGR,...,0,1,0,0,1,0,0,2016-11-30 17:35:33,2016-11-30 18:05:10,0.493611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7727511,A-7776879,2019-08-22 19:48:26,2019-08-22 20:17:11,0.084,At MD-140/Baltimore Blvd - Accident. Lane bloc...,Gamber Rd,Finksburg,Carroll,21048,KDMW,...,1,0,0,0,1,0,0,2019-08-22 19:48:26,2019-08-22 20:17:11,0.479167
7727512,A-7776880,2019-08-22 19:48:26,2019-08-22 20:17:11,0.084,At MD-140/Baltimore Blvd - Accident. Lane bloc...,Emory Rd,Finksburg,Carroll,21048,KDMW,...,1,0,0,0,1,0,0,2019-08-22 19:48:26,2019-08-22 20:17:11,0.479167
7727513,A-7776881,2019-08-22 19:57:25,2019-08-23 00:56:06,3.053,Closed between MD-66/Exit 35 and MD-65/Exit 29...,I-70 W,Hagerstown,Washington,21740,KHGR,...,1,0,0,0,0,0,1,2019-08-22 19:57:25,2019-08-23 00:56:06,4.978056
7727996,A-7777364,2019-08-23 08:03:29,2019-08-23 08:32:31,0.273,At MD-7/Exit 34 - Multi-vehicle accident invol...,I-695 N,Rosedale,Baltimore County,21237,KMTN,...,1,0,0,0,1,0,0,2019-08-23 08:03:29,2019-08-23 08:32:31,0.483889


In [21]:
# Extract accident year, month, and day from start date (I assume no crashes were recorded for longer than 24 hours, so I did not extract
# from both start and end time)
df['Year'], df['Month'], df['Day'] = df['Date'].dt.year, df['Date'].dt.month, df['Date'].dt.day
df

,ID,Start_Time,End_Time,Distance(mi),Description,Street,City,County,Zipcode,Airport_Code,...,Severity_1,Severity_2,Severity_3,Severity_4,Date,End_Date,Accident_Time_Hr,Year,Month,Day
228546,A-228553,2016-11-30 11:58:01,2016-11-30 16:57:20,0.010,Accident on MD-100 Eastbound at Exits 13A 13A-...,I-97 S,Severn,Anne Arundel,21144,KBWI,...,0,1,0,0,2016-11-30 11:58:01,2016-11-30 16:57:20,4.988611,2016.0,11.0,30.0
228547,A-228554,2016-11-30 12:04:03,2016-11-30 17:02:36,0.010,Lane blocked and right hand shoulder blocked d...,I-97 S,Severn,Anne Arundel,21144,KBWI,...,0,0,1,0,2016-11-30 12:04:03,2016-11-30 17:02:36,4.975833,2016.0,11.0,30.0
228560,A-228567,2016-11-30 16:54:53,2016-11-30 17:24:12,0.010,Accident on MD-7 Philadelphia Rd at MD-543 Riv...,Philadelphia Rd,Belcamp,Harford,21017,KAPG,...,0,1,0,0,2016-11-30 16:54:53,2016-11-30 17:24:12,0.488611,2016.0,11.0,30.0
228567,A-228574,2016-11-30 17:33:47,2016-11-30 18:01:49,0.010,Accident on MD-100 near Exits 10 10A 10B MD-71...,MD-100 E,Hanover,Anne Arundel,21076,KBWI,...,0,1,0,0,2016-11-30 17:33:47,2016-11-30 18:01:49,0.467222,2016.0,11.0,30.0
228573,A-228580,2016-11-30 17:35:33,2016-11-30 18:05:10,0.010,Accident on Cleveland Ave around Antietam St.,E Antietam St,Hagerstown,Washington,21740-5701,KHGR,...,0,1,0,0,2016-11-30 17:35:33,2016-11-30 18:05:10,0.493611,2016.0,11.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7727511,A-7776879,2019-08-22 19:48:26,2019-08-22 20:17:11,0.084,At MD-140/Baltimore Blvd - Accident. Lane bloc...,Gamber Rd,Finksburg,Carroll,21048,KDMW,...,0,1,0,0,2019-08-22 19:48:26,2019-08-22 20:17:11,0.479167,2019.0,8.0,22.0
7727512,A-7776880,2019-08-22 19:48:26,2019-08-22 20:17:11,0.084,At MD-140/Baltimore Blvd - Accident. Lane bloc...,Emory Rd,Finksburg,Carroll,21048,KDMW,...,0,1,0,0,2019-08-22 19:48:26,2019-08-22 20:17:11,0.479167,2019.0,8.0,22.0
7727513,A-7776881,2019-08-22 19:57:25,2019-08-23 00:56:06,3.053,Closed between MD-66/Exit 35 and MD-65/Exit 29...,I-70 W,Hagerstown,Washington,21740,KHGR,...,0,0,0,1,2019-08-22 19:57:25,2019-08-23 00:56:06,4.978056,2019.0,8.0,22.0
7727996,A-7777364,2019-08-23 08:03:29,2019-08-23 08:32:31,0.273,At MD-7/Exit 34 - Multi-vehicle accident invol...,I-695 N,Rosedale,Baltimore County,21237,KMTN,...,0,1,0,0,2019-08-23 08:03:29,2019-08-23 08:32:31,0.483889,2019.0,8.0,23.0


In [22]:
# Remove non datetime versions of start and end time columns
df = df.drop(columns=['Start_Time', 'End_Time'])

In [23]:
# Tried to see visually what was left in dataset, spent far too long trying to fuzzy string encode county/city because I saw misspellings of
# st marys, gave up
print(df.apply(lambda col: [col, col.unique()]))

                                                  ID  \
0  228546      A-228553
228547      A-228554
2285...   
1  [A-228553, A-228554, A-228567, A-228574, A-228...   

                                        Distance(mi)  \
0  228546     0.010
228547     0.010
228560     0...   
1  [0.01, 0.0, 1.18, 1.67, 0.68, 0.8, 2.13, 1.61,...   

                                         Description  \
0  228546     Accident on MD-100 Eastbound at Exi...   
1  [Accident on MD-100 Eastbound at Exits 13A 13A...   

                                              Street  \
0  228546              I-97 S
228547             ...   
1  [I-97 S, Philadelphia Rd, MD-100 E, E Antietam...   

                                                City  \
0  228546         Severn
228547         Severn
22...   
1  [Severn, Belcamp, Hanover, Hagerstown, Rockvil...   

                                              County  \
0  228546         Anne Arundel
228547         Ann...   
1  [Anne Arundel, Harford, Washington, Mont

In [24]:
# Noticed some zip codes had 5 digits dash 4 digits, formatted all zip codes to only be first 5 digits as a new column
def fix_zip(series):
      return series.astype(str).str.split('-').str[0].str.zfill(5)

df['Zip'] = fix_zip(df['Zipcode'])

In [25]:
# Removed old non formatted zip code column
df = df.drop(columns='Zipcode')

In [26]:
# Removed id and description, was not sure how to approach either or if either were necessary
df=df.drop(columns=['ID', 'Description'])

In [27]:
# One hot encoded street names
df = onehotinator(df, 'Street')

In [28]:
# One hot encode city/county/airport code
df = onehotinator(df, 'City')
df = onehotinator(df, 'County')
df = onehotinator(df, 'Airport_Code')

In [29]:
# Noticed that somehow at the point of looking at column names and unique values the second time, more null values showed up, 
# deleted them out of hand
df = df.dropna(how='any')
df.isnull().sum()

Distance(mi)         0
Amenity              0
Bump                 0
Crossing             0
Give_Way             0
                    ..
Airport_Code_KRYT    0
Airport_Code_KSBY    0
Airport_Code_KW29    0
Airport_Code_KWAL    0
Airport_Code_KWBC    0
Length: 8027, dtype: int64

In [30]:
df
df.to_csv('Clean_md_data.csv', index=False)

In [32]:
df.shape

(123051, 8027)